In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as rnd
from scipy.spatial.distance import euclidean

In [2]:
def gen_okt(r=1.0, sigma=0.01):
    pos = sigma * rnd.randn(6, 3)
    pos[0, 0] += r
    pos[1, 0] -= r
    pos[2, 1] += r
    pos[3, 1] -= r
    pos[4, 2] += r
    pos[5, 2] -= r
    return pos


def calc_efg(el):
    efg = np.zeros([3, 3], dtype=float)
    for k in el:
        r = np.linalg.norm(k)
        for i in range(3):
            for j in range(3):
                efg[i, j] += -(3 * k[i] * k[j]) / r**5
                if i == j:
                    efg[i, j] += 1 / r**3
    return efg


def get_params(efg):
    ev, evec = np.linalg.eig(efg)
    sort_index = np.argsort(np.abs(ev))
    V_xx = ev[sort_index[0]]
    V_yy = ev[sort_index[1]]
    V_zz = ev[sort_index[2]]
    eta = (V_xx - V_yy) / V_zz
    dirs = evec[sort_index[2]] / np.linalg.norm(evec[sort_index[2]])
    return V_zz, eta, dirs


def run(fun, n=1000000, r=1.0, sigma=0.01):
    Vzz = np.empty(n, dtype=float)
    eta = np.empty(n, dtype=float)
    dirs = np.empty((n, 3), dtype=float)
    for i in range(n):
        pos = fun(r=r, sigma=sigma)
        efg = calc_efg(pos)
        Vzz[i], eta[i], dirs[i] = get_params(efg)
    
    dis = np.empty(n-1, dtype=float)
    for i in range(n-1):
        dis[i] = euclidean(dirs[i], dirs[i+1])
    
    return Vzz, eta, dis

In [28]:
%time Vzz_o, eta_o, dis_o = run(gen_okt)

CPU times: user 2min 26s, sys: 456 ms, total: 2min 27s
Wall time: 2min 27s


In [13]:
e1 = [1, 0, 0]
e2 = [0, 2, 0]
e3 = [-1, 0, 0]
e4 = [0, -2, 0]
e5 = [0, 0, 3]
e6 = [0, 0, -3]
el = [e1, e2, e3, e4, e5, e6]
el

[[1, 0, 0], [0, 2, 0], [-1, 0, 0], [0, -2, 0], [0, 0, 3], [0, 0, -3]]

In [14]:
efg = calc_efg(el)
print(efg)
print(get_params(efg))

[[-3.67592593  0.          0.        ]
 [ 0.          1.57407407  0.        ]
 [ 0.          0.          2.10185185]]
(-3.6759259259259256, 0.14357682619647366, array([1., 0., 0.]))
